#  動的計画法とTD手法


- **[3.1 動的計画法](#3.1-動的計画法)** 
    - **[3.1.1 動的計画法とは](#3.1.1-動的計画法とは)** 
    - **[3.1.2 方策評価](#3.1.2-方策評価)** 
    - **[3.1.3 方策反復](#3.1.3-方策反復)** 
    - **[3.1.4 価値反復](#3.1.4-価値反復)** 
<br><br>
- **[3.2 TD手法](#3.2-TD手法)** 
    - **[3.2.1 TD手法とは](#3.2.1-TD手法とは)** 
    - **[3.2.2 Sarsa](#3.2.2-Sarsa)**
    - **[3.2.3 SarsaにおけるQ関数の実装](#3.2.3-SarsaにおけるQ関数の実装)**
    - **[3.2.4 Sarsaでのε-greedy手法の実装](#3.2.4-Sarsaでのε-greedy手法の実装)**   
    - **[3.2.5 Q学習](#3.2.5-Q学習)**
<br><br>
- **[添削問題](#添削問題)**

***

## 3.1 動的計画法

### 3.1.1 動的計画法とは

ここからは与えられたベルマン方程式を実際に解き、最適な方策を見つけ出すための手法について説明していきます。  
環境のモデルがMDP（マルコフ決定過程）として**完全に**与えられている時の解法アルゴリズムのことを 
**<font color=#AA0000>動的計画法(DP)</font>** と呼称します。そして今回の目的は、ある環境下において最適価値関数または最適行動価値関数を導くことにあります。
***
**マルコフ決定過程とは？** <br>
未来の状態は現在の状態・行動のみによって確率的に決定され、過去の挙動と無関係であるという条件を満たす強化学習過程のことをマルコフ決定過程といいます。<br>

#### 問題


$V^{*}(s) = \max\limits_{\alpha \in A} \displaystyle\sum_{s'\in S}^{} P(s'|s,a)(r(s,a,s') + \gamma V^{*}(s'
))$

$Q^{*}(s,a) = \displaystyle\sum_{s'\in S}^{} P(s'|s,a)(r(s,a,s') + \gamma \max\limits_{\alpha \in A} Q^{*}(s,a))$

上の式は以下のうちどちらに当てはまるでしょうか？


- ベルマン最適方程式
- ベルマン方程式

#### ヒント

- アスタリスクの意味を思い出してみましょう。

#### 解答

- ベルマン最適方程式

***

### 3.1.2 方策評価

まず、最適価値関数を見つけ出す前に、ある特定の手法$\pi$を取った時の価値関数$V^{\pi}(s)$を計算する必要があります。この計算方法を **<font color=#AA0000>方策評価(policy evaluation)</font>** と呼びます。方策評価は以下のように近似します。

>1. 閾値$\epsilon$を事前に設定。更新量が$\epsilon$よりも小さくなるまで計算を行います
>2. 方策$\pi$を入力
>3. $V(s)$を全ての状態において0として仮定   
>4. 以下の動作を繰り返す:  
>    1. $\delta$ = 0  
>    2. 全ての状態sについて :  
>        1. $v = V(s)$
>        2. $V(s) =\displaystyle\sum_{\alpha \in A(s)}^{} \pi(a|s) \displaystyle\sum_{s' \in S}^{} P(s'|s,a)(r(s,a,s') + \gamma V(s'))$ で更新
>        3. $\delta$ = $\max(\delta,|v-V(s)|)$
>    3. $\delta < \epsilon$  ならば、ループを脱出
>5. V(s)を$V^{\pi}$の近似解とみなして出力

4.Aにおいてわざわざ$\delta$をおいているのは、全ての状態の差分を閾値よりも小さくするためです。状態がひとつだけの場合$\delta$を使わずに計算することが可能ですが、状態が複数になると全ての状態の中で差分の最も大きいものを保存しておく変数が必要になります。その役割を$\delta$がみなしています。


#### 問題

・環境1において方策評価を実装し価値関数を求めてください。 <br>
・方策はState_A, State_B両方において常にaction Xを選択するものとします。 <br>
        
<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_7.png">

<center>(図:環境1)</center>

In [ ]:
from markov import *

# 空欄を埋めて価値評価関数を完成させてください
def policy_evaluation(pi, states, epsilon=0.001):

    
    
    
    
    
            return V
        

state_transition = np.array(
    [[0, 0, 0, 0.3, 5],
    [0, 1, 0, 0.7, -10],
    [0, 0, 1, 1, 5],
    [1, 0, 1, 1, 5],
    [1, 2, 0, 0.8, 100],
    [1, 1, 0, 0.2, 1]]
)

states = np.unique(state_transition[:, 0])
actlist = np.unique(state_transition[:, 2])
terminals = [2]
gamma = 0.8

V = {s: 0 for s in np.hstack((states,terminals))}
pi = {s: 0 for s in states}
policy_evaluation(pi, states, epsilon =0.001)

#### ヒント

- Vは `{state : 価値関数の値}` の辞書型で表現してみましょう。
- `policy_evaluation` 関数で渡しているstatesにterminalsの値は含まれていません。Vにはterminalsの状態も必要なので、V作成のときに注意してください。

#### 解答例

In [ ]:
from markov import *

# 空欄を埋めて価値評価関数を完成させてください
def policy_evaluation(pi, states, epsilon=0.001):
    while True:
        delta = 0
        for s in states:
            v = V.copy()
            V[s] = sum([p * (R(s, pi[s], s1) + gamma * V[s1]) for (p, s1) in T(s, pi[s])])
            delta = max(delta, abs(v[s] - V[s]))
        if  delta < epsilon:
            return V
        

state_transition = np.array(
    [[0, 0, 0, 0.3, 5],
    [0, 1, 0, 0.7, -10],
    [0, 0, 1, 1, 5],
    [1, 0, 1, 1, 5],
    [1, 2, 0, 0.8, 100],
    [1, 1, 0, 0.2, 1]]
)

states = np.unique(state_transition[:, 0])
actlist = np.unique(state_transition[:, 2])
terminals = [2]
gamma = 0.8

V = {s: 0 for s in np.hstack((states, terminals))}
pi = {s: 0 for s in states}
policy_evaluation(pi, states, epsilon =0.001)

***

### 3.1.3 方策反復

方策評価ではある手法${\pi}$に対して価値関数$V^{\pi}(s)$を計算しました。  
ここでこのような価値関数においてgreedy手法をとるような方策を$\pi'$と置きます。このような$\pi'$に対して前回の手法を適用し、新たな価値関数$V^{\pi'}(s)$を計算します。すると$V^{\pi} < V^{\pi'}$となることが一般に知られています。これを **<font color=#AA0000>価値改善</font>** と言います。greedy手法とは期待値が大きいと考えられる手法です。<br>

ここではこの価値改善を繰り返していくことを考えましょう。<br>
改善と評価を繰り返すことで最適価値関数を導き出すことが可能になります。これを **<font color=#AA0000>方策反復</font>** といい、以下のアルゴリズムで評価されます。

>1. 全ての状態$s\in S$に対してV(s)と$\pi(s)$を初期化する  
>2. $\delta$が閾値以下になるまで以下を繰り返す:(方策評価)
>    1. $\delta = 0$
>    2. 各状態$s\in S$について:
>        1. $v = V(s)$ 
>        2. $V(s)$ = $\displaystyle\sum_{s'\in S}^{} P(s'|s,\pi(s))(r(s,\pi(s),s') + \gamma V(s'))$で更新
>        3. $\delta$ = $\max(\delta,|v-V(s)|)$ 
>3. policy-flag = True とおく
>4. 各状態$s\in S$について:
>     1. $b = \pi(s)$
>     2. $\pi(s)=arg\max\limits_{a \in A} \displaystyle\sum_{s'\in S}^{} P(s'|s,a)(r(s,a,s') + \gamma V(s'))$
>     3. もし$b \neq$$\pi(s)$ ならば　policy-flag = False
>5. もしpolicy-flag = Trueならば終了。それ以外は2から繰り返す

4.Bは2.の方策評価を行ったあとの価値関数を使って全てのactionにおける価値関数を求めています。二度手間のように感じられますが、4.の方の価値関数はすでに計算されたものであり、方策評価とは少し異なることに注意してください。<br>
4.Bでは方策の更新もしています。

#### 問題

環境1において、空欄を埋めて方策反復関数を実装してください。<br>


<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_7.png">

<center>(図:環境1)</center>

In [ ]:
from markov import *
random.seed(0)

# 方策評価関数です
def policy_evaluation(pi, V, states, epsilon=0.001):
    while True:
        delta = 0
        for s in states:
            v = V.copy()
            V[s] = sum([p * (R(s, pi[s], s1) + gamma * V[s1]) for (p, s1) in T(s,pi[s])])
            delta = max(delta,abs(v[s] - V[s]))
        if  delta < epsilon:
            return V

# 方策反復関数を実装して下さい
def policy_iteration(states):

    

    
    
    
    
    
    
    
    
    
            return pi

state_transition = np.array(
    [[0, 0, 0, 0.3, 5],
    [0, 1, 0, 0.7, -10],
    [0, 0, 1, 1, 5],
    [1, 0, 1, 1, 5],
    [1, 2, 0, 0.8, 100],
    [1, 1, 0, 0.2, 1]]
)

states = np.unique(state_transition[:,0])
terminals = [2]
init = [0]
gamma = 0.8

# 関数を動かして結果を表示します
pi = policy_iteration(states)
print(pi)

#### ヒント

- 上のアルゴリズムの2番は `policy_evaluation` 関数が担っています。
- `pi` の初期値はどんなactionでも大丈夫です。解答例ではランダムに選んでいます。

#### 解答例

In [ ]:
from markov import *
random.seed(0)

# 方策評価関数です
def policy_evaluation(pi, V, states, epsilon=0.001):
    while True:
        delta = 0
        for s in states:
            v = V.copy()
            V[s] = sum([p * (R(s, pi[s], s1) + gamma * V[s1]) for (p, s1) in T(s,pi[s])])
            delta = max(delta,abs(v[s] - V[s]))
        if  delta < epsilon:
            return V

# 方策反復関数を実装して下さい
def policy_iteration(states):
    V = {s: 0 for s in np.hstack((states, terminals))}
    pi = {s: random.choice(actions(s)) for s in states}
    while True:
        V = policy_evaluation(pi, V, states, epsilon=0.001)
        policy_flag = True

        for s in states:
            action =sorted(actions(s), key=lambda a:sum([p * gamma * V[s1] + p * R(s, a, s1) for (p, s1) in T(s, a)]), reverse=True)
            if action[0] != pi[s]:
                pi[s] = action[0]
                policy_flag = False
        if policy_flag:
            return pi

state_transition = np.array(
    [[0, 0, 0, 0.3, 5],
    [0, 1, 0, 0.7, -10],
    [0, 0, 1, 1, 5],
    [1, 0, 1, 1, 5],
    [1, 2, 0, 0.8, 100],
    [1, 1, 0, 0.2, 1]]
)

states = np.unique(state_transition[:,0])
terminals = [2]
init = [0]
gamma = 0.8

# 関数を動かして結果を表示します
pi = policy_iteration(states)
print(pi)

***

### 3.1.4　価値反復

**方策反復**でのアルゴリズムには、毎回全ての状態について価値関数を計算し直す事を複数回しなければならないという問題があります。  
その結果、状態が増えていく場合計算量が大きく増加してしまいます。これを解決するために生み出されたのが **<font color=#AA0000>価値反復</font>** です。以下のようなアルゴリズムで実行されます。

>1. 全ての状態$s\in S$に対してV(s)と$\pi(s)$を初期化する  
>2. 繰り返し
>    1. $\delta = 0$
>    2. 各状態$s\in S$について:
>        1. v = V(s) 
>        2. $V(s) = \max\limits_{\alpha \in A} \displaystyle\sum_{s'\in S}^{} P(s'|s,a)(r(s,a,s') + \gamma V(s'))$で更新
>        3. $\delta$ = max($\delta$,|v-V(s)|)
>        4. $\delta$ < 閾値ならば繰り返しを終了
>3. $\pi = arg \max\limits_{\alpha \in A} \displaystyle\sum_{s'\in S}^{} P(s'|s,a)(r(s,a,s') + \gamma V(s'))$を出力

価値関数の計算が一度で済んでいることがわかると思います。

#### 問題

価値反復関数を実際に実装してみて下さい。

In [ ]:
from markov import *
random.seed(0)

# 方策評価をする関数です
def policy_evaluation(pi, V, states, epsilon=0.001):
    while True:
        delta = 0
        for s in states:
            v = V.copy()
            V[s] = sum([p * (R(s, pi[s], s1) + gamma * V[s1]) for (p, s1) in T(s,pi[s])])
            delta = max(delta,abs(v[s] - V[s]))
        if  delta < epsilon:
            return V
        
# 空欄を埋めて価値反復関数を実装してください
def value_iteration(states, epsilon=0.001):
    V = 
    pi = 
    V = 
    for s in states:
        action =
        pi[s] = 
    return pi


state_transition = np.array(
    [[0, 0, 0, 0.3, 5],
    [0, 1, 0, 0.7, -10],
    [0, 0, 1, 1, 5],
    [1, 0, 1, 1, 5],
    [1, 2, 0, 0.8, 100],
    [1, 1, 0, 0.2, 1]]
)

states = np.unique(state_transition[:, 0])
actlist = np.unique(state_transition[:, 2])
terminals = [2]
init = [0]
gamma = 0.8

pi = value_iteration(states,epsilon=0.001)
print(pi)

#### ヒント

- 実装は方策反復関数とほとんど同じです。

#### 解答例

In [ ]:
from markov import *
random.seed(0)

# 方策評価をする関数です
def policy_evaluation(pi, V, states, epsilon=0.001):
    while True:
        delta = 0
        for s in states:
            v = V.copy()
            V[s] = sum([p * (R(s, pi[s], s1) + gamma * V[s1]) for (p, s1) in T(s,pi[s])])
            delta = max(delta, abs(v[s] - V[s]))
        if  delta < epsilon:
            return V
        
# 空欄を埋めて価値反復関数を実装してください
def value_iteration(states, epsilon=0.001):
    V = {s: 0 for s in np.hstack((states,terminals))}
    pi = {s: random.choice(actions(s)) for s in states}
    V = policy_evaluation(pi, V, states, epsilon=0.001)
    for s in states:
        action = sorted(actions(s), key=lambda a:sum([p * gamma * V[s1] + p * R(s,a,s1) for (p, s1) in T(s, a)]), reverse=True)
        pi[s] = action[0]
    return pi


state_transition = np.array(
    [[0, 0, 0, 0.3, 5],
    [0, 1, 0, 0.7, -10],
    [0, 0, 1, 1, 5],
    [1, 0, 1, 1, 5],
    [1, 2, 0, 0.8, 100],
    [1, 1, 0, 0.2, 1]]
)

states = np.unique(state_transition[:, 0])
actlist = np.unique(state_transition[:, 2])
terminals = [2]
init = [0]
gamma = 0.8

pi = value_iteration(states,epsilon=0.001)
print(pi)

***

## 3.2 TD手法

### 3.2.1 TD手法とは

これまで**動的計画法**について学習してきましたが、非常に大きな欠点が存在します。  
それは状態遷移確率があらかじめわかっていなければならない点です。状態遷移確率がわかっていない状況下では、chapter1で行なったように実際に集めた報酬から状態遷移確率を推定していく必要があります。そのための手法として **<font color=#AA0000>TD手法</font>** が存在します。TDとはTime Differenceの事で、最終結果を見ずに、現在の推定値を利用して次の推定値を更新していく方法です。今回はTD手法の中で有名な手法として
- Sarsa
- Q-learning  

の二つの学習手法を紹介します。

#### 問題

これから環境2を用意して使用します。<br>
<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_8.png">
<center>(図:環境2)</center>
図のような迷路となっており、以下の条件があります。  



- 開始地点 : 図中のstartとあるマス。
- 終端地点 : +1または-1とあるマス。
- 遷移不可能地点 : 黒いマスは通ることができません。また、マスがない箇所には移動できません。
- 移動 : 隣の上下左右に動くことができます。
- 状態遷移確率 : 選択した行動に対して、
     - 8割が行動した通りに遷移します
     - 1割が行動と90度時計回りの方向に遷移します（上を選択すると右に遷移　下を選択すると左に遷移）
     - 1割が行動と90度反時計回りの方向に遷移します（上を選択すると左に遷移　下を選択すると右に遷移）
- 報酬 : +1,-1のマスはそのまま報酬となり、それ以外のマスは全て-0.04であるとします。


    スタート地点から上に２つ進もうとし、右に3つ進もうとした後の位置を出力する関数を作成して下さい。  


In [ ]:
import numpy as np
import random 
np.random.seed(0)

# 新しくエピソードTを再定義しました
def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]

# stateの状態を変化させる関数です
def go(state, direction):
    return [s + d for s,d in zip(state,direction)]

# chapter2.3 報酬と収益 で実装した関数を少し変更して使用します
def take_single_action(state, direction,actions):
    x = np.random.uniform(0,1)
    cumulative_probability = 0.0
    for probability_state in T(state,direction,actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0],next_state[1]]
    if reward is None:
        return state
    else:
        return next_state

# 空欄を埋めて上に２回、右に３回移動する関数を実装してください
def move(s):
    actions = ((1, 0), (0, 1), (-1, 0), (0, -1)) # up,right,down,left
    up = 
    right = 
    for _ in range(2):
        s1 = 
        s = 
    for _ in range(3):
        s2 = 
        s1 = 
    return s2


# 環境を定義します
situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1, None],
                      [None, -0.04, -0.04, -0.04, +1, None],
                      [None, None, None, None, None, None]])
            
terminals=[(2, 4), (3, 4)]
init = [1,1]
# 関数を実行します
move(init)

#### ヒント

- `take_single_action` 関数は次の移動先を返す関数です。
- 繰り返しの動作はfor文を利用して簡略化しましょう。

#### 解答例

In [ ]:
import numpy as np
import random
np.random.seed(0)

# 新しくエピソードTを再定義しました
def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]

# stateの状態を変化させる関数です
def go(state, direction):
    return [s + d for s, d in zip(state, direction)]

# chapter2.3 報酬と収益 で実装した関数を少し変更して使用します
def take_single_action(state, direction,actions):
    x = np.random.uniform(0, 1)
    cumulative_probability = 0.0
    for probability_state in T(state, direction, actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0], next_state[1]]
    if reward is None:
        return state
    else:
        return next_state

# 空欄を埋めて上に２回、右に３回移動する関数を実装してください
def move(s):
    actions = ((1, 0), (0, 1), (-1, 0), (0, -1)) # up,right,down,left
    up = 0
    right = 1
    for _ in range(2):
        s1 = take_single_action(s, up, actions)
        s = s1
    for _ in range(3):
        s2 = take_single_action(s1, right, actions)
        s1 = s2
    return s2


# 環境を定義します
situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1, None],
                      [None, -0.04, -0.04, -0.04, +1, None],
                      [None, None, None, None, None, None]])
            
terminals=[(2, 4), (3, 4)]
init = [1,1]
# 関数を実行します
move(init)

***

### 3.2.2 Sarsa

**Sarsa手法**とはベルマン方程式を試行錯誤しながら解いていくアルゴリズムの一つです。  

ここからはV(s)の代わりに行動価値関数Q(s,a)を使用します。以下のようなアルゴリズムで更新していきます。

>1. Q(s,a)を初期化
>2. エピソードについて繰り返し:
>   1. sを初期化 
>   2. Q関数から導かれる方策($\epsilon$-greedy手法を使用しましょう)を使用して行動を決定する
>   3. エピソード中の時間ステップについて繰り返し:
>       1. 行動aをして遷移した状態s',報酬rを観測する
>       2. Q関数から導かれる方策($\epsilon$-greedy手法を使用しましょう)を使用して次の行動a'を決定する
>       3. $Q(s,a) += \alpha[r + \gamma Q(s',a') - Q(s,a)]$で更新する
>       4. 状態sに遷移した状態s'、行動aに次の行動a'を代入する
>   4. sが終端になれば繰り返しを終了
>3. 予め決めていた目標も満たせばエピソードの繰り返し終了

$\alpha$は学習率で、事前に設定されるパラメーターです。<br>

#### 問題


環境2においてエージェントはstartにいることを考えます。 <br>
ランダムに行動を決定してその時のエピソードの報酬を求めてください。 <br>
- 行動は常に上、右、下、左が選択できるものとします。
- 行動先に壁や障害物があり移動できない場合は、その場にとどまることとし報酬は-0.04になります。
- エピソードの終了条件は報酬が0.7を超えるエピソードが5回連続で続いたときとします。
- エピソード繰り返し数の最大は30回、時間ステップ繰り返し数の最大は100回とします。

 **<font color=#AA0000>今回はランダムに行動しているのでまだQは使用しません。</font>** 


<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_8.png">
<center>(図:環境2)</center>

In [ ]:
import numpy as np
np.random.seed(0)

def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]

def go(state, direction):
    return [s + d for s, d in zip(state, direction)]

# ランダムに次の行動を決定してください
def get_action(t_state, episode):
    # 解答を書いてください
    next_action = 
    return next_action


# chapter2.3 報酬と収益 で実装した関数を少し変更して使用します
# 返し値にrewardが追加されました
def take_single_action(state, direction,actions):
    x = np.random.uniform(0, 1)
    cumulative_probability = 0.0
    for probability_state in T(state,direction,actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0],next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward


num_episodes = 30
max_steps = 100
total_reward = np.zeros(5)
goal_average_reward = 0.7

# 環境を定義します
situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1, None],
                      [None, -0.04, -0.04, -0.04, +1,None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1, 1]
actions = ((1, 0), (0, 1), (-1, 0), (0, -1))


# エピソードの繰り返しを定義します
for episode in range(num_episodes):
    state = init
    # エピソードの繰り返しを定義します
    action = get_action(state, episode)
    episode_reward = 0

    # 時間ステップのループを定義します
    for t in range(max_steps):
        # 空欄を埋めて時間ステップ中の振る舞いを実装してください
        next_state, reward = 
        episode_reward += 
        next_action = 
        state = 
        action = 
        if state in terminals :
            break
            
    #報酬を記録        
    total_reward = np.hstack((total_reward[1:],episode_reward))
    print(total_reward)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    # 直近の100エピソードが規定報酬以上であれば成功
    if (min(total_reward) >= goal_average_reward):
        print('Episode %d train agent successfuly! t=%d' %(episode,t))
        break 

        

#### ヒント

- `get_action` 関数と時間ステップに対応する部分の空欄を埋めて、コードを完成させてください。
- `get_action` では０〜３のどれかをランダムに返すようにコーディングしてください。
- 時間ステップでは次の状態、行動、エピソード中における報酬の合計を計算してください。

#### 解答例

In [ ]:
import numpy as np
np.random.seed(0)

def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]

def go(state, direction):
    return [s + d for s, d in zip(state, direction)]

# ランダムに次の行動を決定してください
def get_action(state, episode):
    next_action = np.random.choice(len(actions))
    return next_action


# chapter2.3 報酬と収益 で実装した関数を少し変更して使用します
# 返し地にrewardが追加されました
def take_single_action(state, direction, actions):
    x = np.random.uniform(0, 1)
    cumulative_probability = 0.0
    for probability_state in T(state, direction, actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0], next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward


num_episodes = 30
max_steps = 100
total_reward = np.zeros(5)
goal_average_reward = 0.7

# 環境を定義します
situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1, None],
                      [None, -0.04, -0.04, -0.04, +1, None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1,1]
actions = ((1,0),(0,1),(-1,0),(0,-1))


# エピソードの繰り返しを定義します
for episode in range(num_episodes): 
    state = init
    # はじめの行動を決定します
    action = get_action(state, episode)
    episode_reward = 0

    # 時間ステップのループを定義します
    for t in range(max_steps):
        # 空欄を埋めて時間ステップ中の振る舞いを実装してください
        next_state, reward = take_single_action(state, action, actions)
        episode_reward += reward
        next_action = get_action(state, episode)
        state = next_state
        action = next_action
        if state in terminals :
            break
            
    #報酬を記録        
    total_reward = np.hstack((total_reward[1:],episode_reward))
    print(total_reward)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    # 直近の100エピソードが規定報酬以上であれば成功
    if (min(total_reward) >= goal_average_reward):
        print('Episode %d train agent successfuly! t=%d' %(episode,t))
        break 
  

***

### 3.2.3 SarsaにおけるQ関数の実装

次は実際に**Q関数**を実装していきます。 <br>
**Q関数**は　[全ての状態×全ての行動]　の配列で表していきます。全ての状態ですが、環境2では取りうる状態は11個(11マスのどこかにいる)、行動は上下左右の4種類です。そこで11×4の配列にしたいのですが、実際の状態は座標で表されておりうまく数字を割り振ることができません。
そこで状態の座標の、<br>
$$ x座標 × 10 + y座標 × 1 $$
をqのインデックスに対応させることとします。 <br>


#### 問題

環境2においてエージェントはstartにいることを考えます。 <br>
Sarsaにてエピソードの報酬を求めてください。 <br>

- 行動の仕方はq関数の値が最も大きいものを採用するとしてください。
- q関数では通常の `state` とは別にq関数用に値を変換した `t_state` という変数を使用します。
- 行動は常に上、右、下、左が選択できるものとします。
- 行動先に壁や障害物があり移動できない場合は、その場にとどまることとし報酬は-0.04になります。
- エピソードの終了条件は報酬が0.7を超えるエピソードが5回連続で続いたときとします。
- エピソード繰り返し数の最大は30回、時間ステップ繰り返し数の最大は100回とします。

q関数更新の際の$\alpha$と$\gamma$は0.4と0.8とします。 <br>


<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_8.png">

<center>(図:環境2)</center>

In [ ]:
import numpy as np
np.random.seed(0)

def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]

def go(state, direction):
    return [s+d for s,d in zip(state,direction)]

# 次の行動を行動価値関数に基づいて決定する関数を実装してください
def get_action(t_state, episode):
    # 解答を書いてください
    next_action = 
    return next_action


def take_single_action(state, direction,actions):
    x = np.random.uniform(0,1)
    cumulative_probability = 0.0
    for probability_state in T(state,direction,actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0],next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward


# 行動価値関数の更新を行う関数を実装してください
def update_qtable(q_table, t_state, action, reward, t_next_state, next_action):
    gamma = 0.8
    alpha = 0.4
    q_table[t_state, action] += 
     
    return q_table

# stateを行動価値関数で使用できるように変換する関数を実装してください
def trans_state(state):
    return 

# 行動価値関数を初期化します
q_table = np.random.uniform(
    low=-0.01, high=0.01, size=(10 ** 2, 4))

num_episodes = 500
max_number_of_steps = 1000
total_reward_vec = np.zeros(5)
goal_average_reward = 0.7

# 環境を定義します
situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1,None],
                      [None, -0.04, -0.04, -0.04, +1,None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1,1]
actions = ((1, 0), (0, 1), (-1, 0), (0, -1))


for episode in range(num_episodes):  
    state = init
    # stateは方向を計算するための変数でt_stateはQ関数に渡すための変数になります
    t_state = trans_state(state)
    action = get_action(t_state, episode)
    episode_reward = 0

    for t in range(max_number_of_steps): 
        next_state, reward = take_single_action(state, action, actions)
        episode_reward += reward 
        next_action = get_action(t_state, episode)
        t_next_state = trans_state(next_state)
        # 行動価値関数を更新してください
        q_table = 
        state = next_state
        t_state = trans_state(state)
        action = next_action
        
        if state in terminals :
            break
    total_reward_vec = np.hstack((total_reward_vec[1:],episode_reward))
    print(total_reward_vec)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    print(min(total_reward_vec),goal_average_reward)
    if (min(total_reward_vec) >= goal_average_reward): 
        print('Episode %d train agent successfuly! t=%d' %(episode, t))
        break 

        

#### ヒント

- `update_qtable` 関数での更新方法は上のアルゴリズムを参照してください。
- `trans_state` 関数では座標を数字に変換する関数を実装します。数字のみを返すようにしてください。

#### 解答例

In [ ]:
import numpy as np
np.random.seed(0)

def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]

def go(state, direction):
    return [s+d for s, d in zip(state, direction)]

# 次の行動を行動価値関数に基づいて決定する関数を実装してください
def get_action(t_state, episode):
    next_action = np.argmax(q_table[t_state])
    return next_action


def take_single_action(state, direction,actions):
    x = np.random.uniform(0,1)
    cumulative_probability = 0.0
    for probability_state in T(state,direction,actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0],next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward


# 行動価値関数の更新を行う関数を実装してください
def update_qtable(q_table, t_state, action, reward, t_next_state, next_action):
    gamma = 0.8
    alpha = 0.4
    q_table[t_state, action] += alpha * (reward + gamma * q_table[t_next_state,next_action] - q_table[t_state,action])
    return q_table

# stateを行動価値関数で使用できるように変換する関数を実装してください
def trans_state(state):
    return sum([n*(10**i) for i, n in enumerate(state)])

# 行動価値関数を初期化します
q_table = np.random.uniform(
    low=-0.01, high=0.01, size=(10 ** 2, 4))

num_episodes = 500
max_number_of_steps = 1000
total_reward_vec = np.zeros(5)
goal_average_reward = 0.7

# 環境を定義します
situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1,None],
                      [None, -0.04, -0.04, -0.04, +1,None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1,1]
actions = ((1, 0), (0, 1), (-1, 0), (0, -1))


for episode in range(num_episodes):  
    state = init
    # stateは方向を計算するための変数でt_stateはQ関数に渡すための変数になります
    t_state = trans_state(state)
    action = get_action(t_state, episode)
    episode_reward = 0

    for t in range(max_number_of_steps): 
        next_state, reward = take_single_action(state, action, actions)
        episode_reward += reward
        next_action = get_action(t_state, episode)
        t_next_state = trans_state(next_state)
        # 行動価値関数を更新してください
        q_table = update_qtable(q_table, t_state, action, reward, t_next_state, next_action)
        state = next_state
        t_state = trans_state(state)
        action = next_action
        
        if state in terminals :
            break
    total_reward_vec = np.hstack((total_reward_vec[1:], episode_reward)) 
    print(total_reward_vec)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    print(min(total_reward_vec),goal_average_reward)
    if (min(total_reward_vec) >= goal_average_reward):
        print('Episode %d train agent successfuly! t=%d' %(episode, t))
        break 

        

***

### 3.2.4 Sarsaでのε-greedy 手法の実装

先程の例ではQ関数の値が大きいもののみを選択していました。<br>
ただこれではより優れた方法を見逃してしまう恐れがあります。そこで **$\epsilon$-greedy手法** を用いることによって新しい手法を探索していきます。
chapter1で行った$\epsilon$-greedy手法とは少し変えて、$\epsilon$の値をepisodeの早い段階では探索の割合を大きくし、episodeが進むに連れ探索を狭めていきます。今回は、 <br>
$$\epsilon = 0.5 * (1 / ( episode + 1)) $$
で計算してみましょう。

#### 問題

環境2においてエージェントはstartにいることを考えます。 <br>
Sarsaにてエピソードの報酬を求めてください。 <br>

- 行動の仕方は$\epsilon$-greedy手法を利用したq関数の値が最も大きいものを採用するとしてください。
- エピソード繰り返し数の最大は200回、時間ステップ繰り返し数の最大は100回とします。
- 行動は常に上、右、下、左が選択できるものとします。
- 行動先に壁や障害物があり移動できない場合は、その場にとどまることとし報酬は-0.04になります。
- エピソードの終了条件は報酬が0.7を超えるエピソードが5回連続で続いたときとします。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_8.png">
<center>(図:環境2)</center>

In [ ]:
import numpy as np
np.random.seed(0)


def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]


def go(state, direction):
    return [s+d for s, d in zip(state, direction)]

# 空欄を埋めてepsilon-greedy手法により次の行動を決定してください
def get_action(t_state, episode):
    epsilon = 
    if epsilon <= 
        next_action = 
    else:
        next_action = 
    return next_action



def take_single_action(state, direction,actions):
    x = np.random.uniform(0,1)
    cumulative_probability = 0.0
    for probability_state in T(state,direction,actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0], next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward



def update_Qtable(q_table, t_state, action, reward, t_next_state, next_action):
    gamma = 0.8
    alpha = 0.4
    q_table[t_state, action] += alpha * (reward + gamma * q_table[t_next_state, next_action] - q_table[t_state, action])
    return q_table

def trans_state(state):
    return [n*(11**i) for i, n in enumerate(state)][0]

q_table = np.random.uniform(
    low=-0.01, high=0.01, size=(10 ** 2, 4))

num_episodes = 500
max_number_of_steps = 1000
total_reward_vec = np.zeros(5)
goal_average_reward = 0.7

situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1,None],
                      [None, -0.04, -0.04, -0.04, +1,None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1,1]
actions = ((1,0),(0,1),(-1,0),(0,-1))
state = [n*(11**i) for i, n in enumerate(init)]


for episode in range(num_episodes):
    state = init
    t_state = trans_state(state)
    action = np.argmax(q_table[t_state])
    episode_reward = 0

    for t in range(max_number_of_steps): 
        next_state, reward = take_single_action(state, action, actions)
        episode_reward += reward  
        next_action = get_action(t_state, episode)
        t_next_state = trans_state(next_state)
        q_table = update_Qtable(q_table, t_state, action, reward, t_next_state, next_action)
        state = next_state
        t_state = trans_state(state)
        action = next_action
        
        if state in terminals :
            break
    total_reward_vec = np.hstack((total_reward_vec[1:],episode_reward))  
    print(total_reward_vec)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    print(min(total_reward_vec),goal_average_reward)
    if (min(total_reward_vec) >= goal_average_reward): 
        print('Episode %d train agent successfuly! t=%d' %(episode, t))
        break

#### ヒント

- $\epsilon$の値は $\epsilon$ = $0.5*\frac{1}{episode + 1}$ になります。

#### 解答例

In [ ]:
import numpy as np
np.random.seed(0)


def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]


def go(state, direction):
    return [s+d for s, d in zip(state, direction)]

# 空欄を埋めてepsilon-greedy手法により次の行動を決定してください
def get_action(t_state, episode):
    epsilon = 0.5 * (1 / (episode + 1))
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[t_state])
    else:
        next_action = np.random.choice(len(actions))
    return next_action



def take_single_action(state, direction, actions):
    x = np.random.uniform(0, 1)
    cumulative_probability = 0.0
    for probability_state in T(state, direction, actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0], next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward



def update_Qtable(q_table, t_state, action, reward, t_next_state, next_action):
    gamma = 0.8
    alpha = 0.4
    q_table[t_state, action] += alpha * (reward + gamma * q_table[t_next_state, next_action] - q_table[t_state, action])
    return q_table

def trans_state(state):
    return sum([n*(10**i) for i, n in enumerate(state)])

q_table = np.random.uniform(
    low=-0.01, high=0.01, size=(10 ** 2, 4))

num_episodes = 500
max_number_of_steps = 1000
total_reward_vec = np.zeros(5)
goal_average_reward = 0.7

situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1, None],
                      [None, -0.04, -0.04, -0.04, +1, None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1,1]
actions = ((1, 0), (0, 1), (-1, 0), (0, -1))
state = [n*(10**i) for i,n in enumerate(init)]

for episode in range(num_episodes):
    state = init
    t_state = trans_state(state)
    action = np.argmax(q_table[t_state])
    episode_reward = 0

    for t in range(max_number_of_steps): 
        next_state, reward = take_single_action(state, action, actions)
        episode_reward += reward  
        next_action = get_action(t_state, episode)
        t_next_state = trans_state(next_state)
        q_table = update_Qtable(q_table, t_state, action, reward, t_next_state, next_action)
        state = next_state
        t_state = trans_state(state)
        action = next_action
        
        if state in terminals :
            break
    total_reward_vec = np.hstack((total_reward_vec[1:],episode_reward))  
    print(total_reward_vec)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    print(min(total_reward_vec),goal_average_reward)
    if (min(total_reward_vec) >= goal_average_reward): 
        print('Episode %d train agent successfuly! t=%d' %(episode, t))
        break 

***

### 3.2.5 Q学習

別の有名なTD手法として**Q学習**があります。 <br>
Q学習も全体の流れはSarsaと変わりなく、 <br>
1. q関数の更新を行ったあとに次の行動を決定すること
2. q関数の更新の方法  

のみが異なります。

流れとしては、
>1. Q(s,a)を初期化
>2. エピソードについて繰り返し:
>   1. sを初期化 
>   2. Q関数から導かれる方策($\epsilon$-greedy手法を使用しましょう)を使用して行動を決定する
>   3. エピソード中の時間ステップについて繰り返し:
>       1. 行動aをして遷移した状態s',報酬rを観測する
>       2. $Q(s,a) +=  \alpha [r + \gamma \max\limits_{a'\in A} Q(s',a')-Q(s,a)]$で更新する
>       3. Q関数から導かれる方策($\epsilon$-greedy手法を使用しましょう)を使用して次の行動a'を決定する
>       4. 状態sに遷移した状態s'、行動aに次の行動a'を代入する
>   4. sが終端になれば繰り返しを終了
>3. 予め決めていた目標も満たせばエピソードの繰り返し終了

$\epsilon$の値は $\epsilon$ = $0.5*\frac{1}{episode + 1}$ になります。

$\alpha$は学習率を示しています。 <br>

流れを見てもSarsaとほとんど変わりないことがわかっていただけると思います。 <br>

Sarsaではq関数を実際次に行う行動を使って更新していましたが、Q学習では次に考えられる行動のうち最もq関数が大きいものを採用します。

#### 問題

SarsaとQ学習の説明として適切なものを選択してください。

- SarsaはTD手法のため状態遷移確率が必要ないが、Q学習では必要である。
- SersaとQ学習において異なるのはq関数の更新の方法のみである。
- Q学習では更新の際に使用した行動と、実際に行う行動が異なることがある。
- 上の3つの選択肢の中に適切なものは存在しない。

#### ヒント

- Q学習では次の行動を決定する前にQ関数の更新を行います。<br>
そのため更新前は最適だと思われた行動も、更新後には他の行動が最適になることがあります。

#### 解答

- Q学習では更新の際に使用した行動と、実際に行う行動が異なることがある。

***

## 添削問題

#### 問題

環境2においてエージェントはstartにいることを考えます。 <br>
Q学習を用いてエピソードの報酬を求めてください。 <br>

- 行動の仕方はq関数の値が最も大きいものを採用するとしてください。
- q関数では通常のstateとは別にq関数用に値を変換したt_stateという変数を使用します。
- 行動は常に上、右、下、左が選択できるものとします。
- 行動先に壁や障害物があり移動できない場合は、その場にとどまることとし報酬は-0.04になります。
- エピソードの終了条件は報酬が0.7を超えるエピソードが5回連続で続いたときとします。
- エピソード繰り返し数の最大は30回、時間ステップ繰り返し数の最大は100回とします。

q関数更新の際の$\alpha$と$\gamma$は0.4と0.8とします。 <br>

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/_RL/img_8.png">
<center>(図:環境2)</center>

In [ ]:
import numpy as np
np.random.seed(0)


def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]


def go(state, direction):
    return [s+d for s, d in zip(state, direction)]

def get_action(t_state, episode):
    epsilon = 0.5 * (1 / (episode + 1))
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[t_state])
    else:
        next_action = np.random.choice(len(actions))
    return next_action


def take_single_action(state, direction, actions):
    x = np.random.uniform(0,1)
    cumulative_probability = 0.0
    for probability_state in T(state, direction, actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0], next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward



def update_Qtable(q_table, t_state, action, reward, t_next_state, next_action):
    gamma = 0.8
    alpha = 0.4
    # Q学習の方法でq関数を完成させてください。
    
    
    return q_table

def trans_state(state):
    return sum([n*(10**i) for i, n in enumerate(state)])

q_table = np.random.uniform(
    low=-0.01, high=0.01, size=(10 ** 2, 4))

num_episodes = 500
max_number_of_steps = 1000
total_reward_vec = np.zeros(5)
goal_average_reward = 0.7

situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1,None],
                      [None, -0.04, -0.04, -0.04, +1,None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1,1]
actions = ((1, 0), (0, 1), (-1, 0), (0, -1))
state = [n*(10**i) for i, n in enumerate(init)]

for episode in range(num_episodes):
    state = init
    t_state = trans_state(state)
    action = np.argmax(q_table[t_state])
    episode_reward = 0
    
    # 以下の空欄を埋めてコードを完成させてください。
    for t in range(max_number_of_steps): 

        
        
        
        if state in terminals :
            break
    total_reward_vec = np.hstack((total_reward_vec[1:],episode_reward))  
    print(total_reward_vec)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    print(min(total_reward_vec),goal_average_reward)
    if (min(total_reward_vec) >= goal_average_reward): 
        print('Episode %d train agent successfuly! t=%d' %(episode, t))
        break 

        

#### ヒント

- Q学習ではq関数の更新を行ったあとに、次の行動を決定します。
- q関数の更新にはまずstateにおけるq関数の最大値を求めてください。

#### 解答例

In [ ]:
import numpy as np
np.random.seed(0)


def T(state, direction, actions):
        return [(0.8, go(state, actions[direction])),
                (0.1, go(state, actions[(direction + 1) % 4])),
                (0.1, go(state, actions[(direction - 1) % 4]))]


def go(state, direction):
    return [s+d for s, d in zip(state, direction)]

def get_action(t_state, episode):
    epsilon = 0.5 * (1 / (episode + 1))
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[t_state])
    else:
        next_action = np.random.choice(len(actions))
    return next_action


def take_single_action(state, direction,actions):
    x = np.random.uniform(0, 1)
    cumulative_probability = 0.0
    for probability_state in T(state, direction, actions):
        probability, next_state = probability_state
        cumulative_probability += probability
        if x < cumulative_probability:
            break
    reward = situation[next_state[0], next_state[1]]
    if reward is None:
        return state, -0.04
    else:
        return next_state, reward



def update_Qtable(q_table, t_state, action, reward, t_next_state):
    gamma = 0.8
    alpha = 0.4
    # Q学習の方法でq関数を完成させてください。
    next_max_q = max(q_table[t_next_state])
    q_table[t_state, action] += alpha * (reward + gamma*next_max_q - q_table[t_state, action] )
    return q_table

def trans_state(state):
    return sum([n*(10**i) for i, n in enumerate(state)])

q_table = np.random.uniform(
    low=-0.01, high=0.01, size=(10 ** 2, 4))

num_episodes = 500
max_number_of_steps = 1000
total_reward_vec = np.zeros(5)
goal_average_reward = 0.7

situation = np.array([[None, None, None, None, None, None],
                      [None, -0.04, -0.04, -0.04, -0.04, None],
                      [None, -0.04, None, -0.04, -1,None],
                      [None, -0.04, -0.04, -0.04, +1,None],
                      [None, None, None, None, None, None]])
            

terminals=[[2, 4], [3, 4]]
init = [1,1]
actions = ((1, 0), (0, 1), (-1, 0), (0, -1))
state = [n*(10**i) for i,n in enumerate(init)]

for episode in range(num_episodes):
    state = init
    t_state = trans_state(state)
    action = np.argmax(q_table[t_state])
    episode_reward = 0
    
    # 以下の空欄を埋めてコードを完成させてください。
    for t in range(max_number_of_steps): 
        next_state, reward = take_single_action(state, action, actions)
        episode_reward += reward  
        t_next_state = trans_state(next_state)
        q_table = update_Qtable(q_table, t_state, action, reward, t_next_state)
        state = next_state
        next_action = get_action(t_state, episode)
        t_state = trans_state(state)
        action = next_action
        
        if state in terminals :
            break
    total_reward_vec = np.hstack((total_reward_vec[1:], episode_reward))  
    print(total_reward_vec)
    print("Episode %d has finished. t=%d" %(episode+1, t+1))
    print(min(total_reward_vec),goal_average_reward)
    if (min(total_reward_vec) >= goal_average_reward): 
        print('Episode %d train agent successfuly! t=%d' %(episode, t))
        break 

        

***